In [1]:
!pip install wandb

In [2]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

  Cloning https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to /tmp/pip-req-build-t61ljfy9
  Running command git clone --filter=blob:none --quiet https://github.com/ildoonet/pytorch-gradual-warmup-lr.git /tmp/pip-req-build-t61ljfy9
  Resolved https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to commit 6b5e8953a80aef5b324104dc0c2e9b8c34d622bd
  Preparing metadata (setup.py) ... done


In [3]:
import pandas as pd
import numpy as np
import random

from tqdm import tqdm
import wandb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler

import torchaudio
import torchaudio.transforms as T
from torchaudio.transforms import AmplitudeToDB, Vol

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
wandb.login()

wandb: Currently logged in as: darya-dare. Use `wandb login --relogin` to force relogin


True

### Custom Dataset

In [5]:
class AudioDataset(Dataset):

    def __init__(self, path_speech, path_noise, eps_value=1e-6,\
                 mel_nframes=300, sample_rate=16000, n_fft=480,\
                 hop_length=160, n_mels=40):

        self.speech = pd.read_csv(path_speech)
        self.noise = pd.read_csv(path_noise)
        self.eps_value = eps_value
        self.mel_nframes = mel_nframes
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.mel_spectrogram = T.MelSpectrogram(sample_rate=self.sample_rate,
                                                n_fft=self.n_fft,
                                                hop_length=self.hop_length,
                                                center=True,
                                                pad_mode="reflect",
                                                power=2.0,
                                                norm='slaney',
                                                n_mels=self.n_mels)

    def __len__(self):

        return len(self.noise)

    def get_same_shape(self, waveform):

        melsp = torch.log(self.mel_spectrogram(waveform) + self.eps_value)
        has_mel_nframes = melsp.size(2) if melsp.size(2) < self.mel_nframes\
         else self.mel_nframes

        image_tens = torch.full((1, self.n_mels, self.mel_nframes), melsp.min())
        image_tens[0, :, :has_mel_nframes] = melsp[:, :, :has_mel_nframes]

        return image_tens

    def load_item(self, df, idx, w_type):

        sample_wav_file = df['filename'][idx]
        wave, _ = torchaudio.load(sample_wav_file)
        wave = wave.float()

        if w_type==1:
            wave_noise = wave.numpy()
            wave_noise_no_silence = [float(el) for el in wave_noise[0] if abs(el) >= 0.001]
            wave = torch.tensor([wave_noise_no_silence])

        wave_melsp = self.get_same_shape(wave)

        return wave_melsp

    def __getitem__(self, idx):

        wave_speech = self.load_item(self.speech, idx, w_type=0)

        luck = random.random()
        if luck > 0.5:
            wave_noise = self.load_item(self.noise, idx, w_type=1)

            wave_out = wave_speech + wave_noise
            label = 1

        else:
            wave_out = wave_speech
            label = 0

        return wave_out, torch.tensor(label)

In [6]:
audio_dataset_train = AudioDataset('./CleanSpeechTrain.csv',
                                   './NoiseTrain.csv')

audio_dataset_val = AudioDataset('./CleanSpeechVal.csv',
                                   './NoiseVal.csv')

audio_dataset_test = AudioDataset('./CleanSpeechTest.csv',
                                   './NoiseTest.csv')

#### Data Loader

In [7]:
train_loader = DataLoader(dataset=audio_dataset_train,
                          batch_size=64, drop_last=True, num_workers=6)

val_loader = DataLoader(dataset=audio_dataset_val,
                        batch_size=64, drop_last=True, num_workers=6)

test_loader = DataLoader(dataset=audio_dataset_test,
                         batch_size=64, drop_last=True, num_workers=6)

### Train functions

In [8]:
def compute_acc_loss(model, loader, loss):

    model.eval()

    correct_samples = 0
    total_samples = 0
    loss_accum = 0
    num_batches = 0

    for i, (x, y) in enumerate(loader):

        x_gpu = x.to(device)
        y_gpu = y.to(device)
        
        prediction = model(x_gpu).squeeze(1)
        loss_value = loss(prediction, y_gpu.float())
        
        correct_samples += torch.sum(prediction.round()==y_gpu)
        total_samples += y.size(0)
        loss_accum += loss_value.item()
        
        num_batches += 1

    return float(correct_samples)/total_samples, loss_accum/num_batches

In [9]:
def train(model, train_loader, val_loader, loss,
          optimizer, warmup_scheduler, num_epochs):

    wandb.init(
        project='Noise-Estimation',
        config={'learning_rate': 1e-5,
                'model': 'ResNet101',
                'dataset': 'original',
                'loss': 'BCELoss',
                'optimizer': 'SGD',
                'scheduler': 'WarmUp and CosineAnnealing',
                'T_max': 60,
                'total_epochs': 60,
                'epochs': 150,
                'batch_size': 64}
    )

    loss_history = []
    train_history = []
    val_history = []

    for epoch in range(num_epochs):
        model.train()

        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        num_batches = 0

        with tqdm(total=len(train_loader),
                  desc=f'Epoch {epoch+1}',
                  leave=True) as pb:

            for i, (x, y) in enumerate(train_loader):

                x_gpu = x.to(device)
                y_gpu = y.to(device)

                optimizer.zero_grad()
                prediction = model(x_gpu).squeeze(1)

                loss_value = loss(prediction, y_gpu.float())
                loss_value.backward()
                optimizer.step()

                correct_samples += torch.sum(prediction.round()==y_gpu)
                total_samples += y.size(0)
                loss_accum += loss_value

                pb.update()
                pb.set_description( f'Epoch {epoch+1}:'
                                    f' Average epoch loss: {loss_accum / num_batches:.3f}'
                                    f' Train accuracy:'
                                    f' {correct_samples/total_samples:.2f},'
                                    )
                num_batches += 1

            ave_loss = loss_accum / num_batches
            train_accuracy = float(correct_samples) / total_samples
            val_accuracy, val_loss = compute_acc_loss(model, val_loader, loss)

            if warmup_scheduler:
                warmup_scheduler.step()

            loss_history.append(float(ave_loss))
            train_history.append(train_accuracy)
            val_history.append(val_accuracy)

            wandb.log({'Epoch': epoch+1, 'loss': ave_loss,
                       'train_acc': train_accuracy,
                       'val_loss': val_loss,
                       'val_acc': val_accuracy,})

            pb.set_description( f'Epoch {epoch+1}:'
                                f' Average epoch loss: {ave_loss:.3f}'
                                f' Train accuracy: {train_accuracy:.2f}'
                                f' Val loss: {val_loss:.3f}'
                                f' Val accuracy: {val_accuracy:.2f}'
                                )

    wandb.finish()
    return loss_history, train_history, val_history

### ResNet50

In [10]:
from ResNet import ResNet50

model_rn = ResNet50(1, device)
model_rn.type(torch.cuda.FloatTensor)
model_rn.to(device)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_rn.parameters(), lr=1e-2)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=20,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-02.


In [11]:
loss_history, train_history, val_history = train(model_rn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler,
                                                 num_epochs=50)

Epoch 1: Average epoch loss: 0.688 Train accuracy: 0.50 Val loss: 0.682 Val accuracy: 0.51: 100%|██████████| 48/48 [00:38<00:00,  1.23it/s]
Epoch 2: Average epoch loss: 0.616 Train accuracy: 0.67 Val loss: 0.375 Val accuracy: 0.96: 100%|██████████| 48/48 [00:41<00:00,  1.17it/s]
Epoch 3: Average epoch loss: 0.500 Train accuracy: 0.85 Val loss: 0.359 Val accuracy: 0.76: 100%|██████████| 48/48 [00:41<00:00,  1.17it/s]
Epoch 4: Average epoch loss: 0.320 Train accuracy: 0.93 Val loss: 0.501 Val accuracy: 0.60: 100%|██████████| 48/48 [00:39<00:00,  1.21it/s]
Epoch 5: Average epoch loss: 0.175 Train accuracy: 0.97 Val loss: 0.139 Val accuracy: 0.99: 100%|██████████| 48/48 [00:38<00:00,  1.25it/s]
Epoch 6: Average epoch loss: 0.108 Train accuracy: 0.98 Val loss: 0.102 Val accuracy: 1.00: 100%|██████████| 48/48 [00:39<00:00,  1.22it/s]
Epoch 7: Average epoch loss: 0.060 Train accuracy: 0.99 Val loss: 0.043 Val accuracy: 1.00: 100%|██████████| 48/48 [00:40<00:00,  1.20it/s]
Epoch 8: Average epo

KeyboardInterrupt: 

### Testing ResNet50

In [12]:
rn_test_accuracy, rn_test_loss = compute_acc_loss(model_rn, test_loader, loss)
print(f'BCResNet model test accuracy: {rn_test_accuracy:.2f}')
print(f'BCResNet model test loss: {rn_test_loss:.3f}')

BCResNet model test accuracy: 1.00
BCResNet model test loss: 0.001


In [17]:
torch.save(model_rn.state_dict(), 'weights_rn50.pth')

In [18]:
torch.save(optimizer.state_dict(), 'optimizer_rn50.pth')

### Testing ResNet50 on the phone audio recordings (in Russian)

In [16]:
rn_phone_test_accuracy, rn_phone_test_loss = compute_acc_loss(model_rn, phone_loader, loss)
print(f'BCResNet model test accuracy: {rn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {rn_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.33
BCResNet model test loss: 5.397


### ResNet101

In [10]:
from ResNet import ResNet101

model_rn = ResNet101(1, device)
model_rn.type(torch.cuda.FloatTensor)
model_rn.to(device)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_rn.parameters(), lr=1e-5)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=60, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=60,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-05.


In [11]:
loss_history, train_history, val_history = train(model_rn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler,
                                                 num_epochs=150)

Epoch 1: Average epoch loss: 0.697 Train accuracy: 0.53 Val loss: 0.697 Val accuracy: 0.53: 100%|██████████| 48/48 [00:42<00:00,  1.13it/s]
Epoch 2: Average epoch loss: 0.695 Train accuracy: 0.54 Val loss: 0.696 Val accuracy: 0.56: 100%|██████████| 48/48 [00:41<00:00,  1.15it/s]
Epoch 3: Average epoch loss: 0.691 Train accuracy: 0.56 Val loss: 0.695 Val accuracy: 0.54: 100%|██████████| 48/48 [00:40<00:00,  1.17it/s]
Epoch 4: Average epoch loss: 0.698 Train accuracy: 0.53 Val loss: 0.695 Val accuracy: 0.55: 100%|██████████| 48/48 [00:43<00:00,  1.09it/s]
Epoch 5: Average epoch loss: 0.692 Train accuracy: 0.56 Val loss: 0.691 Val accuracy: 0.55: 100%|██████████| 48/48 [00:44<00:00,  1.08it/s]
Epoch 6: Average epoch loss: 0.689 Train accuracy: 0.57 Val loss: 0.693 Val accuracy: 0.55: 100%|██████████| 48/48 [00:42<00:00,  1.13it/s]
Epoch 7: Average epoch loss: 0.690 Train accuracy: 0.56 Val loss: 0.689 Val accuracy: 0.57: 100%|██████████| 48/48 [00:43<00:00,  1.11it/s]
Epoch 8: Average epo

Adjusting learning rate of group 0 to 9.9931e-06.


Epoch 63: Average epoch loss: 0.517 Train accuracy: 0.92 Val loss: 0.497 Val accuracy: 0.92: 100%|██████████| 48/48 [00:42<00:00,  1.13it/s]


Adjusting learning rate of group 0 to 9.9726e-06.


Epoch 64: Average epoch loss: 0.519 Train accuracy: 0.90 Val loss: 0.494 Val accuracy: 0.93: 100%|██████████| 48/48 [00:42<00:00,  1.12it/s]


...

Epoch 149: Average epoch loss: 0.435 Train accuracy: 0.90 Val loss: 0.417 Val accuracy: 0.93: 100%|██████████| 48/48 [00:44<00:00,  1.09it/s]


Adjusting learning rate of group 0 to 4.4774e-06.


Epoch 150: Average epoch loss: 0.427 Train accuracy: 0.92 Val loss: 0.420 Val accuracy: 0.92: 100%|██████████| 48/48 [00:43<00:00,  1.10it/s]

Adjusting learning rate of group 0 to 4.7383e-06.


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,███▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▃▄▄▄▃▄▄▅▅▆▇▇█████▇███████████████████
val_acc,▁▁▁▃▃▃▄▃▃▄▄▅▆▇▇████▇▇█████▇█████████████
val_loss,███▇▇▇▆▆▆▅▅▅▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▂▂▂▁▁▁▁▁▁▁▁
Epoch,150
loss,0.42662
train_acc,0.91992
val_acc,0.91504
val_loss,0.42042


### Testing Resnet101

In [12]:
rn_test_accuracy, rn_test_loss = compute_acc_loss(model_rn, test_loader, loss)
print(f'BCResNet model test accuracy: {rn_test_accuracy:.2f}')
print(f'BCResNet model test loss: {rn_test_loss:.3f}')

BCResNet model test accuracy: 0.91
BCResNet model test loss: 0.425


### Testing ResNet101 on the phone audio recordings (in Russian)

In [18]:
rn_phone_test_accuracy, rn_phone_test_loss = compute_acc_loss(model_rn, phone_loader, loss)
print(f'BCResNet model test accuracy: {rn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {rn_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.50
BCResNet model test loss: 0.747


### ResNet152

In [10]:
from ResNet import ResNet152

model_rn = ResNet152(1, device)
model_rn.type(torch.cuda.FloatTensor)
model_rn.to(device)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_rn.parameters(), lr=1e-3)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=40,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-03.


In [11]:
loss_history, train_history, val_history = train(model_rn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler,
                                                 num_epochs=80)

Epoch 1: Average epoch loss: 0.798 Train accuracy: 0.48 Val loss: 0.803 Val accuracy: 0.47: 100%|██████████| 96/96 [00:49<00:00,  1.94it/s]
Epoch 2: Average epoch loss: 0.721 Train accuracy: 0.44 Val loss: 0.507 Val accuracy: 0.60: 100%|██████████| 96/96 [00:48<00:00,  1.98it/s]
Epoch 3: Average epoch loss: 0.657 Train accuracy: 0.52 Val loss: 0.475 Val accuracy: 0.53: 100%|██████████| 96/96 [00:48<00:00,  1.99it/s]
Epoch 4: Average epoch loss: 0.636 Train accuracy: 0.58 Val loss: 0.387 Val accuracy: 0.61: 100%|██████████| 96/96 [00:47<00:00,  2.00it/s]
Epoch 5: Average epoch loss: 0.607 Train accuracy: 0.71 Val loss: 0.420 Val accuracy: 0.94: 100%|██████████| 96/96 [00:48<00:00,  1.99it/s]
Epoch 6: Average epoch loss: 0.623 Train accuracy: 0.62 Val loss: 0.420 Val accuracy: 0.82: 100%|██████████| 96/96 [00:48<00:00,  1.98it/s]
Epoch 7: Average epoch loss: 0.640 Train accuracy: 0.59 Val loss: 0.614 Val accuracy: 0.68: 100%|██████████| 96/96 [00:49<00:00,  1.94it/s]
Epoch 8: Average epo

Adjusting learning rate of group 0 to 9.9846e-04.


Epoch 43: Average epoch loss: 0.019 Train accuracy: 0.99 Val loss: 0.010 Val accuracy: 1.00: 100%|██████████| 96/96 [00:48<00:00,  1.97it/s]


Adjusting learning rate of group 0 to 9.9384e-04.


Epoch 44: Average epoch loss: 0.021 Train accuracy: 0.99 Val loss: 0.008 Val accuracy: 1.00: 100%|██████████| 96/96 [00:47<00:00,  2.01it/s]


...

Adjusting learning rate of group 0 to 6.5451e-04.


Epoch 58: Average epoch loss: 0.006 Train accuracy: 1.00 Val loss: 0.002 Val accuracy: 1.00: 100%|██████████| 96/96 [00:48<00:00,  2.00it/s]


Adjusting learning rate of group 0 to 6.1672e-04.


Epoch 59:   0%|          | 0/96 [00:01<?, ?it/s]


KeyboardInterrupt: 

### Testing ResNet152

In [14]:
rn_test_accuracy, rn_test_loss = compute_acc_loss(model_rn, test_loader, loss)
print(f'ResNet152 model test accuracy: {rn_test_accuracy:.2f}')
print(f'ResNet152 model test loss: {rn_test_loss:.3f}')

ResNet152 model test accuracy: 1.00
ResNet152 model test loss: 0.002


### Saving ResNet152 weights and optimizer

In [23]:
torch.save(model_rn.state_dict(), 'weights_rn152.pth')

In [24]:
torch.save(optimizer.state_dict(), 'optimizer_rn152.pth')

### Testing ResNet152 on the phone audio recordings (in Russian)

In [12]:
rn_test_accuracy, rn_test_loss = compute_acc_loss(model_rn, test_loader, loss)
print(f'BCResNet model test accuracy: {rn_test_accuracy:.2f}')
print(f'BCResNet model test loss: {rn_test_loss:.3f}')

BCResNet model test accuracy: 1.00
BCResNet model test loss: 0.002


### Custom Test (in English) vor ResNet Model

In [18]:
rn_test_accuracy, rn_test_loss = compute_acc_loss(model_rn, test_loader, loss)
print(f'VGG model test accuracy: {rn_test_accuracy:.2f}')
print(f'VGG model test loss: {rn_test_loss:.3f}')

VGG model test accuracy: 1.00
VGG model test loss: 0.017


### Custom Test (in Russian) vor ResNet Model

In [22]:
rn_test_accuracy, rn_test_loss = compute_acc_loss(model_rn, rtest_loader, loss)
print(f'VGG model test accuracy: {rn_test_accuracy:.2f}')
print(f'VGG model test loss: {rn_test_loss:.3f}')

VGG model test accuracy: 1.00
VGG model test loss: 0.007


### MHAttKWS

In [17]:
from BCResNet import MHAttKWS
model_mhat = MHAttKWS(num_classes=1)
model_mhat.type(torch.cuda.FloatTensor)
model_mhat.to(device)

loss = nn.BCEWithLogitsLoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_mhat.parameters(), lr=1e-2)
cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=40,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-02.


In [18]:
loss_history, train_history, val_history = train(model_mhat, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler,
                                                 num_epochs=50)

Epoch,▁▂▃▄▅▆▇█
loss,█▄▄▃▂▁▁▁
train_acc,█▃▄▅▄▂▇▁
val_acc,█▁▁▄▅▅▃▃
Epoch,8
loss,0.69293
train_acc,0.49574
val_acc,0.499


Epoch 1: Average epoch loss: 0.693 Train accuracy: 0.49, Val accuracy: 0.50: 100%|██████████| 61/61 [00:54<00:00,  1.12it/s]
Epoch 2: Average epoch loss: 0.693 Train accuracy: 0.49, Val accuracy: 0.51: 100%|██████████| 61/61 [00:52<00:00,  1.16it/s]
Epoch 3: Average epoch loss: 0.695 Train accuracy: 0.52, Val accuracy: 0.50: 100%|██████████| 61/61 [00:53<00:00,  1.13it/s]
Epoch 4: Average epoch loss: 0.694 Train accuracy: 0.51, Val accuracy: 0.52: 100%|██████████| 61/61 [00:52<00:00,  1.15it/s]
Epoch 5: Average epoch loss: 0.694 Train accuracy: 0.50, Val accuracy: 0.47: 100%|██████████| 61/61 [00:54<00:00,  1.13it/s]
Epoch 6: Average epoch loss: 0.693 Train accuracy: 0.49, Val accuracy: 0.51: 100%|██████████| 61/61 [00:54<00:00,  1.12it/s]
Epoch 7: Average epoch loss: 0.694 Train accuracy: 0.50, Val accuracy: 0.53: 100%|██████████| 61/61 [00:52<00:00,  1.16it/s]
Epoch 8: Average epoch loss: 0.693 Train accuracy: 0.49, Val accuracy: 0.49: 100%|██████████| 61/61 [00:55<00:00,  1.11it/s]


Adjusting learning rate of group 0 to 9.9846e-03.


Epoch 43: Average epoch loss: 0.690 Train accuracy: 0.50, Val accuracy: 0.51: 100%|██████████| 61/61 [00:52<00:00,  1.15it/s]


Adjusting learning rate of group 0 to 9.9384e-03.


...

Epoch 49: Average epoch loss: 0.687 Train accuracy: 0.49, Val accuracy: 0.51: 100%|██████████| 61/61 [00:54<00:00,  1.11it/s]


Adjusting learning rate of group 0 to 9.0451e-03.


Epoch 50: Average epoch loss: 0.687 Train accuracy: 0.50, Val accuracy: 0.49: 100%|██████████| 61/61 [00:53<00:00,  1.14it/s]

Adjusting learning rate of group 0 to 8.8020e-03.


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▇▇█▇▆▇▆▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▂▂▁
train_acc,▄▄█▆▄▅▄▆▅▇▇▅▅▃▄▃▆▆▆▇▇▆▅▄▆▇▆▇▇▇▁▄█▃▆▇▆▅▆▅
val_acc,▅▆▅▇▅█▄▂█▅█▄▁▂▃▄▆▆▆▄▅▃▅█▅▅▄▆▃▄▂▅▅▃▅▃▅▂▅▃
Epoch,50
loss,0.68664
train_acc,0.49902
val_acc,0.488


### BCResNet: scale=1

In [10]:
from BCResNet import BCResNet
model_bcrn = BCResNet(num_labels=1)
model_bcrn.type(torch.cuda.FloatTensor)
model_bcrn.to(device)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_bcrn.parameters(), lr=1e-3)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=25, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=25,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-03.


In [11]:
loss_history, train_history, val_history = train(model_bcrn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler, num_epochs=100)

Epoch 1: Average epoch loss: 0.692 Train accuracy: 0.52 Val loss: 0.697 Val accuracy: 0.48: 100%|██████████| 48/48 [00:37<00:00,  1.29it/s]
Epoch 2: Average epoch loss: 0.695 Train accuracy: 0.48 Val loss: 0.695 Val accuracy: 0.51: 100%|██████████| 48/48 [00:38<00:00,  1.25it/s]
Epoch 3: Average epoch loss: 0.691 Train accuracy: 0.52 Val loss: 0.697 Val accuracy: 0.49: 100%|██████████| 48/48 [00:39<00:00,  1.23it/s]
Epoch 4: Average epoch loss: 0.691 Train accuracy: 0.52 Val loss: 0.698 Val accuracy: 0.48: 100%|██████████| 48/48 [00:39<00:00,  1.22it/s]
Epoch 5: Average epoch loss: 0.693 Train accuracy: 0.50 Val loss: 0.695 Val accuracy: 0.51: 100%|██████████| 48/48 [00:38<00:00,  1.24it/s]
Epoch 6: Average epoch loss: 0.691 Train accuracy: 0.52 Val loss: 0.694 Val accuracy: 0.50: 100%|██████████| 48/48 [00:38<00:00,  1.25it/s]
Epoch 7: Average epoch loss: 0.691 Train accuracy: 0.51 Val loss: 0.694 Val accuracy: 0.50: 100%|██████████| 48/48 [00:38<00:00,  1.25it/s]
Epoch 8: Average epo

Adjusting learning rate of group 0 to 9.9606e-04.


Epoch 28: Average epoch loss: 0.627 Train accuracy: 0.83 Val loss: 0.632 Val accuracy: 0.87: 100%|██████████| 48/48 [00:37<00:00,  1.26it/s]


Adjusting learning rate of group 0 to 9.8429e-04.


Epoch 29: Average epoch loss: 0.618 Train accuracy: 0.84 Val loss: 0.625 Val accuracy: 0.88: 100%|██████████| 48/48 [00:38<00:00,  1.25it/s]


...

Epoch 99: Average epoch loss: 0.123 Train accuracy: 0.99 Val loss: 0.072 Val accuracy: 1.00: 100%|██████████| 48/48 [00:37<00:00,  1.28it/s]


Adjusting learning rate of group 0 to 1.5708e-05.


Epoch 100: Average epoch loss: 0.127 Train accuracy: 0.99 Val loss: 0.071 Val accuracy: 1.00: 100%|██████████| 48/48 [00:37<00:00,  1.27it/s]

Adjusting learning rate of group 0 to 3.9426e-06.


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██████████▇▇▇▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▄▄▃▃▂▂▁▁▁▁▁▁
train_acc,▁▁▁▁▂▂▃▃▄▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
val_acc,▁▁▁▁▁▁▁▂▂▂▅▆▇███████████████████████████
val_loss,██████████▇▇▇▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▄▄▃▂▂▂▁▁▁▁▁▁
Epoch,100
loss,0.12697
train_acc,0.98926
val_acc,1.0
val_loss,0.07119


In [23]:
torch.save(model_bcrn.state_dict(), 'weights_bcrn100ep.pth')

In [24]:
torch.save(optimizer.state_dict(), 'optimizer_bcrn100ep.pth')

### Testing BCResNet

In [12]:
bcrn_test_accuracy, bcrn_test_loss = compute_acc_loss(model_bcrn, test_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_test_accuracy:.2f}')
print(f'BCResNet model test loss: {bcrn_test_loss:.3f}')

BCResNet model test accuracy: 1.00
BCResNet model test loss: 0.070


### Custom Test (in English) for BCResNet model

In [64]:
from BCResNet import BCResNet
bcrn_model = BCResNet(num_labels=1)
bcrn_model.load_state_dict(torch.load('./weights_bcrn1.pth'))
bcrn_model.type(torch.FloatTensor)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.Adam(bcrn_model.parameters(),lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_bcrn1.pth'))

In [66]:
test_dataset = TestDataset('./CleanSpeechCustomTest.csv',
                            './NoiseTest.csv')

In [67]:
test_loader = DataLoader(dataset=test_dataset, batch_size=4, 
                         drop_last=True)

In [68]:
bcrn_loaded_test_accuracy, vgg_loaded_test_loss = compute_acc_loss(bcrn_model, test_loader, loss)
print(f'BCResNet model test accuracy: {vgg_loaded_test_accuracy:.2f}')
print(f'BCResNet model test loss: {vgg_loaded_test_loss:.3f}')

BCResNet model test accuracy: 0.90
BCResNet model test loss: 0.000


### Custom Test (in Russian) for BCResNet model

In [69]:
from BCResNet import BCResNet
bcrn_model = BCResNet(num_labels=1)
bcrn_model.load_state_dict(torch.load('./weights_bcrn1.pth'))
bcrn_model.type(torch.FloatTensor)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.Adam(bcrn_model.parameters(),lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_bcrn1.pth'))

In [71]:
rtest_dataset = RTestDataset('./CleanSpeechCustomTestRus.csv',
                             './NoiseTest.csv')

In [72]:
rtest_loader = DataLoader(dataset=rtest_dataset, batch_size=2, 
                          drop_last=True)

In [73]:
bcrn_loaded_test_accuracy, vgg_loaded_test_loss = compute_acc_loss(bcrn_model, rtest_loader, loss)
print(f'BCResNet model test accuracy: {vgg_loaded_test_accuracy:.2f}')
print(f'BCResNet model test loss: {vgg_loaded_test_loss:.3f}')

BCResNet model test accuracy: 0.90
BCResNet model test loss: 0.000


### Testing BCResNet on phone audio recordings (in Russian)

In [13]:
class PhoneDataset(Dataset):

    def __init__(self, path_speech, path_noise, eps_value=1e-6,\
                 mel_nframes=300, sample_rate=16000, n_fft=480,\
                 hop_length=160, n_mels=40):

        self.speech = pd.read_csv(path_speech)
        self.noise = pd.read_csv(path_noise)
        self.eps_value = eps_value
        self.mel_nframes = mel_nframes
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.mel_spectrogram = T.MelSpectrogram(sample_rate=self.sample_rate,
                                                n_fft=self.n_fft,
                                                hop_length=self.hop_length,
                                                center=True,
                                                pad_mode="reflect",
                                                power=2.0,
                                                norm='slaney',
                                                n_mels=self.n_mels)

    def __len__(self):

        return len(self.speech)

    def get_same_shape(self, waveform):

        melsp = torch.log(self.mel_spectrogram(waveform) + self.eps_value)
        has_mel_nframes = melsp.size(2) if melsp.size(2) < self.mel_nframes\
         else self.mel_nframes

        image_tens = torch.full((1, self.n_mels, self.mel_nframes), melsp.min())
        image_tens[0, :, :has_mel_nframes] = melsp[:, :, :has_mel_nframes]

        return image_tens

    def load_item(self, df, idx):

        sample_wav_file = df['filename'][idx]
        wave, _ = torchaudio.load(sample_wav_file)
        wave = wave.float()

        wave_melsp = self.get_same_shape(wave)

        return wave_melsp

    def __getitem__(self, idx):
        luck = random.random()
        
        if luck > 0.5:
            wave_out = self.load_item(self.speech, idx)
            label = 0
        else:
            wave_out = self.load_item(self.noise, idx)
            label = 1

        return wave_out, torch.tensor(label)

In [14]:
phone_dataset = PhoneDataset('./CleanSpeechCustomTest.csv',
                            './NoiseTest.csv')

In [15]:
phone_loader = DataLoader(dataset=phone_dataset, batch_size=2, 
                          drop_last=True)

In [21]:
from BCResNet import BCResNet
bcrn_model = BCResNet(num_labels=1)
bcrn_model.load_state_dict(torch.load('./weights_bcrn1.pth'))
bcrn_model.type(torch.FloatTensor)
bcrn_model.to(device)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.SGD(bcrn_model.parameters(),lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_bcrn1.pth'))

In [22]:
bcrn_phone_test_accuracy, vgg_phone_test_loss = compute_acc_loss(bcrn_model, phone_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {vgg_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.67
BCResNet model test loss: 2.592


In [22]:
bcrn_phone_test_accuracy, bcrn_phone_test_loss = compute_acc_loss(model_bcrn, phone_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {bcrn_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.50
BCResNet model test loss: 2.750


In [6]:
from BCResNet import BCResNet
bcrn_model = BCResNet(num_labels=1)
bcrn_model.load_state_dict(torch.load('./weights_bcrn.pth'))
bcrn_model.type(torch.FloatTensor)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.Adam(bcrn_model.parameters(),lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_bcrn.pth'))

In [7]:
bcrn_phone_test_accuracy, vgg_phone_test_loss = compute_acc_loss(bcrn_model, phone_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {vgg_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.39
BCResNet model test loss: 8.904


### BCResNet scale=1 on the reduced dataset

In [10]:
audio_dataset_train = AudioDataset('./CleanSpeechTrainRN.csv',
                                   './NoiseTrain.csv')

audio_dataset_val = AudioDataset('./CleanSpeechValRN.csv',
                                   './NoiseVal.csv')

audio_dataset_test = AudioDataset('./CleanSpeechTestRN.csv',
                                   './NoiseTest.csv')

In [11]:
train_loader = DataLoader(dataset=audio_dataset_train,
                          batch_size=32, drop_last=True, num_workers=6)

val_loader = DataLoader(dataset=audio_dataset_val,
                        batch_size=32, drop_last=True, num_workers=6)

test_loader = DataLoader(dataset=audio_dataset_test,
                         batch_size=32, drop_last=True, num_workers=6)

In [10]:
from BCResNet import BCResNet
model_bcrn = BCResNet(num_labels=1)
model_bcrn.type(torch.cuda.FloatTensor)
model_bcrn.to(device)

loss = nn.BCELoss()
# .type(torch.cuda.FloatTensor)
optimizer = optim.Adam(model_bcrn.parameters(), lr=1e-3)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=20,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-03.


In [12]:
loss_history, train_history, val_history = train(model_bcrn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler, num_epochs=50)

In [12]:
torch.save(model_bcrn.state_dict(), 'weights_bcrn.pth')

In [13]:
torch.save(optimizer.state_dict(), 'optimizer_bcrn.pth')

### Testing BCResNet scale=1 on the reduced dataset

In [14]:
from BCResNet import BCResNet
bcrn_model = BCResNet(num_labels=1)
bcrn_model.load_state_dict(torch.load('./weights_bcrn.pth'))
bcrn_model.type(torch.FloatTensor)
bcrn_model.to(device)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.Adam(bcrn_model.parameters(),lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_bcrn.pth'))

In [15]:
bcrn_test_accuracy, bcrn_test_loss = compute_acc_loss(bcrn_model, test_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_test_accuracy:.2f}')
print(f'BCResNet model test loss: {bcrn_test_loss:.3f}')

BCResNet model test accuracy: 1.00
BCResNet model test loss: 0.001


### Testing BCResNet scale=1 (trained on the reduced dataset) on the phone audio recordings (in Russian)

In [19]:
from BCResNet import BCResNet
bcrn_model = BCResNet(num_labels=1)
bcrn_model.load_state_dict(torch.load('./weights_bcrn.pth'))
bcrn_model.type(torch.FloatTensor)
bcrn_model.to(device)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.Adam(bcrn_model.parameters(),lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_bcrn.pth'))

In [20]:
bcrn_phone_test_accuracy, bcrn_phone_test_loss = compute_acc_loss(bcrn_model, phone_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {bcrn_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.46
BCResNet model test loss: 6.423


### BCResNet: scale=1, optimizer=SGD, reduced dataset 

In [8]:
from BCResNet import BCResNet
model_bcrn = BCResNet(num_labels=1)
model_bcrn.type(torch.cuda.FloatTensor)
model_bcrn.to(device)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_bcrn.parameters(), lr=1e-3)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=20,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-03.


In [12]:
loss_history, train_history, val_history = train(model_bcrn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler, num_epochs=50)

Epoch 1: Average epoch loss: 0.703 Train accuracy: 0.49 Val loss: 0.671 Val accuracy: 0.50: 100%|██████████| 96/96 [00:40<00:00,  2.36it/s]
Epoch 2: Average epoch loss: 0.697 Train accuracy: 0.52 Val loss: 0.675 Val accuracy: 0.49: 100%|██████████| 96/96 [00:39<00:00,  2.42it/s]
Epoch 3: Average epoch loss: 0.703 Train accuracy: 0.49 Val loss: 0.667 Val accuracy: 0.50: 100%|██████████| 96/96 [00:40<00:00,  2.35it/s]
Epoch 4: Average epoch loss: 0.695 Train accuracy: 0.51 Val loss: 0.670 Val accuracy: 0.48: 100%|██████████| 96/96 [00:39<00:00,  2.45it/s]
Epoch 5: Average epoch loss: 0.691 Train accuracy: 0.51 Val loss: 0.664 Val accuracy: 0.50: 100%|██████████| 96/96 [00:39<00:00,  2.44it/s]
Epoch 6: Average epoch loss: 0.692 Train accuracy: 0.50 Val loss: 0.659 Val accuracy: 0.51: 100%|██████████| 96/96 [00:39<00:00,  2.41it/s]
Epoch 7: Average epoch loss: 0.687 Train accuracy: 0.51 Val loss: 0.656 Val accuracy: 0.52: 100%|██████████| 96/96 [00:40<00:00,  2.34it/s]
Epoch 8: Average epo

Adjusting learning rate of group 0 to 9.9384e-04.


Epoch 23: Average epoch loss: 0.352 Train accuracy: 0.98 Val loss: 0.283 Val accuracy: 0.99: 100%|██████████| 96/96 [00:37<00:00,  2.54it/s]


Adjusting learning rate of group 0 to 9.7553e-04.


Epoch 24: Average epoch loss: 0.305 Train accuracy: 0.99 Val loss: 0.228 Val accuracy: 1.00: 100%|██████████| 96/96 [00:38<00:00,  2.50it/s]


...

Epoch 49: Average epoch loss: 0.079 Train accuracy: 1.00 Val loss: 0.075 Val accuracy: 1.00: 100%|██████████| 96/96 [00:38<00:00,  2.46it/s]


Adjusting learning rate of group 0 to 3.4549e-04.


Epoch 50: Average epoch loss: 0.083 Train accuracy: 0.99 Val loss: 0.067 Val accuracy: 1.00: 100%|██████████| 96/96 [00:37<00:00,  2.54it/s]

Adjusting learning rate of group 0 to 4.2178e-04.


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█████████▇▇▇▇▇▆▆▅▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▁▁▁▁▂▂▃▄▅▆▆▇▇▇████████████████████████
val_acc,▁▁▁▁▁▁▁▁▁▄▅▇▇███████████████████████████
val_loss,█████████▇▇▇▇▇▆▆▅▄▄▃▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,50
loss,0.08311
train_acc,0.99251
val_acc,0.99805
val_loss,0.06658


### Testing and saving BCResNet scale=1, trained on the reduced dataset, optimizer=SGD

In [13]:
bcrn_test_accuracy, bcrn_test_loss = compute_acc_loss(model_bcrn, test_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_test_accuracy:.2f}')
print(f'BCResNet model test loss: {bcrn_test_loss:.3f}')

BCResNet model test accuracy: 1.00
BCResNet model test loss: 0.074


In [14]:
torch.save(model_bcrn.state_dict(), 'weights_bcrn.pth')

In [15]:
torch.save(optimizer.state_dict(), 'optimizer_bcrn.pth')

### Testing BCResNet scale=1 (trained on the reduced dataset, optimizer=SGD) on phone audio recordings (in Russian)

In [24]:
from BCResNet import BCResNet
bcrn_model = BCResNet(num_labels=1)
bcrn_model.load_state_dict(torch.load('./weights_bcrn.pth'))
bcrn_model.type(torch.FloatTensor)
bcrn_model.to(device)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.SGD(bcrn_model.parameters(),lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_bcrn.pth'))

In [25]:
bcrn_phone_test_accuracy, vgg_phone_test_loss = compute_acc_loss(bcrn_model, phone_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {vgg_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.50
BCResNet model test loss: 2.398


### BCResNet: scale=8

In [12]:
from BCResNet import BCResNet
model_bcrn = BCResNet(scale=8, num_labels=1)
model_bcrn.type(torch.cuda.FloatTensor)
model_bcrn.to(device)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_bcrn.parameters(), lr=1e-3)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=40,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-03.


In [13]:
loss_history, train_history, val_history = train(model_bcrn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler, num_epochs=50)

Epoch 1: Average epoch loss: 0.696 Train accuracy: 0.49 Val loss: 0.694 Val accuracy: 0.49: 100%|██████████| 154/154 [00:39<00:00,  3.91it/s]
Epoch 2: Average epoch loss: 0.694 Train accuracy: 0.50 Val loss: 0.695 Val accuracy: 0.46: 100%|██████████| 154/154 [00:39<00:00,  3.87it/s]
Epoch 3: Average epoch loss: 0.692 Train accuracy: 0.53 Val loss: 0.690 Val accuracy: 0.52: 100%|██████████| 154/154 [00:40<00:00,  3.79it/s]
Epoch 4: Average epoch loss: 0.689 Train accuracy: 0.56 Val loss: 0.685 Val accuracy: 0.60: 100%|██████████| 154/154 [00:39<00:00,  3.92it/s]
Epoch 5: Average epoch loss: 0.682 Train accuracy: 0.63 Val loss: 0.672 Val accuracy: 0.88: 100%|██████████| 154/154 [00:39<00:00,  3.89it/s]
Epoch 6: Average epoch loss: 0.674 Train accuracy: 0.68 Val loss: 0.664 Val accuracy: 0.96: 100%|██████████| 154/154 [00:40<00:00,  3.80it/s]
Epoch 7: Average epoch loss: 0.667 Train accuracy: 0.70 Val loss: 0.654 Val accuracy: 0.98: 100%|██████████| 154/154 [00:39<00:00,  3.86it/s]
Epoch 

Adjusting learning rate of group 0 to 9.9846e-04.


Epoch 43: Average epoch loss: 0.019 Train accuracy: 1.00 Val loss: 0.015 Val accuracy: 1.00: 100%|██████████| 154/154 [00:39<00:00,  3.88it/s]


Adjusting learning rate of group 0 to 9.9384e-04.


Epoch 44: Average epoch loss: 0.015 Train accuracy: 1.00 Val loss: 0.002 Val accuracy: 1.00: 100%|██████████| 154/154 [00:39<00:00,  3.85it/s]


...

Epoch 49: Average epoch loss: 0.017 Train accuracy: 1.00 Val loss: 0.004 Val accuracy: 1.00: 100%|██████████| 154/154 [00:39<00:00,  3.93it/s]


Adjusting learning rate of group 0 to 9.0451e-04.


Epoch 50: Average epoch loss: 0.015 Train accuracy: 1.00 Val loss: 0.010 Val accuracy: 1.00: 100%|██████████| 154/154 [00:39<00:00,  3.91it/s]

Adjusting learning rate of group 0 to 8.8020e-04.


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,████████▇▇▇▇▆▅▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▂▄▄▄▅▆▆▇▇████████████████████████████
val_acc,▁▁▂▃▇█████▇▇▇▇▇▇▇███████████████████████
val_loss,██████▇▇▇▇▇▇▆▅▅▄▄▂▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,50
loss,0.01535
train_acc,0.99675
val_acc,0.99804
val_loss,0.00967


### Testing BCResNet scale=8

In [14]:
bcrn_test_accuracy, bcrn_test_loss = compute_acc_loss(model_bcrn, test_loader, loss)
print(f'BCResNet scale=8 model test accuracy: {bcrn_test_accuracy:.2f}')
print(f'BCResNet scale=8 model test loss: {bcrn_test_loss:.3f}')

BCResNet scale=8 model test accuracy: 1.00
BCResNet scale=8 model test loss: 0.007


In [20]:
torch.save(model_bcrn.state_dict(), 'weights_bcrn_scale8.pth')

In [21]:
torch.save(optimizer.state_dict(), 'optimizer_bcrn_scale8.pth')

### Testing BCResNet scale=8 on the phone audio recordings (in Russian)

In [19]:
bcrn_phone_test_accuracy, bcrn_phone_test_loss = compute_acc_loss(model_bcrn, phone_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {bcrn_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.47
BCResNet model test loss: 5.357


### BCResNet: scale=16

In [10]:
from BCResNet import BCResNet
model_bcrn = BCResNet(scale=16, num_labels=1)
model_bcrn.type(torch.cuda.FloatTensor)
model_bcrn.to(device)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_bcrn.parameters(), lr=1e-3)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=40,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-03.


In [11]:
loss_history, train_history, val_history = train(model_bcrn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler, num_epochs=50)

Epoch 1: Average epoch loss: 0.699 Train accuracy: 0.50 Val loss: 0.693 Val accuracy: 0.51: 100%|██████████| 154/154 [00:56<00:00,  2.74it/s]
Epoch 2: Average epoch loss: 0.693 Train accuracy: 0.52 Val loss: 0.691 Val accuracy: 0.51: 100%|██████████| 154/154 [00:59<00:00,  2.59it/s]
Epoch 3: Average epoch loss: 0.689 Train accuracy: 0.55 Val loss: 0.684 Val accuracy: 0.52: 100%|██████████| 154/154 [00:59<00:00,  2.57it/s]
Epoch 4: Average epoch loss: 0.679 Train accuracy: 0.64 Val loss: 0.675 Val accuracy: 0.72: 100%|██████████| 154/154 [00:59<00:00,  2.60it/s]
Epoch 5: Average epoch loss: 0.670 Train accuracy: 0.71 Val loss: 0.662 Val accuracy: 0.95: 100%|██████████| 154/154 [00:59<00:00,  2.60it/s]
Epoch 6: Average epoch loss: 0.660 Train accuracy: 0.70 Val loss: 0.650 Val accuracy: 0.99: 100%|██████████| 154/154 [00:59<00:00,  2.60it/s]
Epoch 7: Average epoch loss: 0.647 Train accuracy: 0.76 Val loss: 0.637 Val accuracy: 0.98: 100%|██████████| 154/154 [00:59<00:00,  2.60it/s]
Epoch 

Adjusting learning rate of group 0 to 9.9846e-04.


Epoch 43: Average epoch loss: 0.019 Train accuracy: 0.99 Val loss: 0.039 Val accuracy: 0.99: 100%|██████████| 154/154 [00:59<00:00,  2.59it/s]


Adjusting learning rate of group 0 to 9.9384e-04.


Epoch 44: Average epoch loss: 0.015 Train accuracy: 1.00 Val loss: 0.012 Val accuracy: 1.00: 100%|██████████| 154/154 [00:59<00:00,  2.59it/s]


...

Epoch 49: Average epoch loss: 0.015 Train accuracy: 1.00 Val loss: 0.004 Val accuracy: 1.00: 100%|██████████| 154/154 [01:00<00:00,  2.56it/s]


Adjusting learning rate of group 0 to 9.0451e-04.


Epoch 50: Average epoch loss: 0.017 Train accuracy: 0.99 Val loss: 0.003 Val accuracy: 1.00: 100%|██████████| 154/154 [00:59<00:00,  2.57it/s]

Adjusting learning rate of group 0 to 8.8020e-04.


Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█████▇▇▇▇▆▆▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▂▃▄▅▅▆▇███████████████████████████████
val_acc,▁▁▁▄███▇▇▇▇▇▇▇██████████████████████████
val_loss,█████▇▇▇▆▆▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,50
loss,0.01706
train_acc,0.99383
val_acc,1.0
val_loss,0.00295


### Testing BCResNet scale=16

In [12]:
bcrn_test_accuracy, bcrn_test_loss = compute_acc_loss(model_bcrn, test_loader, loss)
print(f'BCResNet scale=8 model test accuracy: {bcrn_test_accuracy:.2f}')
print(f'BCResNet scale=8 model test loss: {bcrn_test_loss:.3f}')

BCResNet scale=8 model test accuracy: 1.00
BCResNet scale=8 model test loss: 0.005


In [17]:
torch.save(model_bcrn.state_dict(), 'weights_bcrn_scale16.pth')

In [18]:
torch.save(optimizer.state_dict(), 'optimizer_bcrn_scale16.pth')

### Testing BCResNet scale=16 on the phone audio recordings (in Russian)

In [16]:
bcrn_phone_test_accuracy, bcrn_phone_test_loss = compute_acc_loss(model_bcrn, phone_loader, loss)
print(f'BCResNet model test accuracy: {bcrn_phone_test_accuracy:.2f}')
print(f'BCResNet model test loss: {bcrn_phone_test_loss:.3f}')

BCResNet model test accuracy: 0.47
BCResNet model test loss: 7.151


### VGG

In [5]:
from vggNet import VGGModel

model_vgg = VGGModel(1, 'vgg11')
model_vgg.to(device)
model_vgg.type(torch.cuda.FloatTensor)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_vgg.parameters(), lr=1e-3)
cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=25,
                                          after_scheduler=cosine_scheduler)

Adjusting learning rate of group 0 to 1.0000e-03.


In [12]:
loss_history, train_history, val_history = train(model_vgg, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler,
                                                 num_epochs=80)

wandb: Currently logged in as: darya-dare. Use `wandb login --relogin` to force relogin


Epoch 1: Average epoch loss: 0.743 Train accuracy: 0.52, Val accuracy: 0.49: 100%|██████████| 30/30 [00:57<00:00,  1.92s/it]
Epoch 2: Average epoch loss: 0.714 Train accuracy: 0.51, Val accuracy: 0.51: 100%|██████████| 30/30 [00:57<00:00,  1.92s/it]
Epoch 3: Average epoch loss: 0.695 Train accuracy: 0.51, Val accuracy: 0.50: 100%|██████████| 30/30 [00:55<00:00,  1.85s/it]
Epoch 4: Average epoch loss: 0.682 Train accuracy: 0.53, Val accuracy: 0.50: 100%|██████████| 30/30 [00:59<00:00,  2.00s/it]
Epoch 5: Average epoch loss: 0.679 Train accuracy: 0.52, Val accuracy: 0.48: 100%|██████████| 30/30 [00:57<00:00,  1.92s/it]
Epoch 6: Average epoch loss: 0.678 Train accuracy: 0.52, Val accuracy: 0.51: 100%|██████████| 30/30 [00:54<00:00,  1.83s/it]
Epoch 7: Average epoch loss: 0.658 Train accuracy: 0.54, Val accuracy: 0.50: 100%|██████████| 30/30 [00:58<00:00,  1.97s/it]
Epoch 8: Average epoch loss: 0.659 Train accuracy: 0.54, Val accuracy: 0.51: 100%|██████████| 30/30 [00:59<00:00,  1.97s/it]


Adjusting learning rate of group 0 to 9.9901e-04.


Epoch 28: Average epoch loss: 0.201 Train accuracy: 0.94, Val accuracy: 0.98: 100%|██████████| 30/30 [00:56<00:00,  1.87s/it]


Adjusting learning rate of group 0 to 9.9606e-04.


...

Epoch 79: Average epoch loss: 0.012 Train accuracy: 1.00, Val accuracy: 1.00: 100%|██████████| 30/30 [00:58<00:00,  1.95s/it]


Adjusting learning rate of group 0 to 8.8564e-06.


Epoch 80: Average epoch loss: 0.012 Train accuracy: 1.00, Val accuracy: 1.00: 100%|██████████| 30/30 [00:55<00:00,  1.86s/it]

Adjusting learning rate of group 0 to 1.5708e-05.


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▇▇▇▇▆▆▅▅▄▄▃▃▂▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▁▁▁▂▂▂▃▅▆▆▆▆▇▇█▆███████████████████████
val_acc,▂▂▁▂▁▂▂▆▅▇▇▂▇███████████████████████████
Epoch,80
loss,0.01185
train_acc,0.998
val_acc,0.999


In [13]:
torch.save(model_vgg.state_dict(), 'weights_vgg1.pth')

In [14]:
torch.save(optimizer.state_dict(), 'optimizer_vgg1.pth')

### Testing VGG

In [15]:
vgg_test_accuracy = compute_acc_loss(model_vgg, test_loader)
print(f'VGG model test accuracy: {vgg_test_accuracy:.2f}')

VGG model test accuracy: 1.00


In [9]:
from vggNet import VGGModel

vgg_model = VGGModel(1, 'vgg11')
vgg_model.load_state_dict(torch.load('./weights_vgg1.pth'))
vgg_model.type(torch.FloatTensor)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.SGD(vgg_model.parameters(), lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_vgg1.pth'))

In [10]:
vgg_loaded_test_accuracy, vgg_loaded_test_loss = compute_acc_loss(vgg_model, test_loader, loss)
print(f'VGG model test accuracy: {vgg_loaded_test_accuracy:.2f}')
print(f'VGG model test loss: {vgg_loaded_test_loss:.3f}')

VGG model test accuracy: 0.99
VGG model test loss: 0.011


### Custom Test (in English) vor VGG Model

In [15]:
class TestDataset(Dataset):

    def __init__(self, path_speech, path_noise, eps_value=1e-6,\
                 mel_nframes=300, sample_rate=16000, n_fft=480,\
                 hop_length=160, n_mels=40):

        self.speech = pd.read_csv(path_speech)
        self.noise = pd.read_csv(path_noise)
        self.eps_value = eps_value
        self.mel_nframes = mel_nframes
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.mel_spectrogram = T.MelSpectrogram(sample_rate=self.sample_rate,
                                                n_fft=self.n_fft,
                                                hop_length=self.hop_length,
                                                center=True,
                                                pad_mode="reflect",
                                                power=2.0,
                                                norm='slaney',
                                                n_mels=self.n_mels)

    def __len__(self):

        return len(self.speech)

    def get_same_shape(self, waveform):

        melsp = torch.log(self.mel_spectrogram(waveform) + self.eps_value)
        has_mel_nframes = melsp.size(2) if melsp.size(2) < self.mel_nframes\
         else self.mel_nframes

        image_tens = torch.full((1, self.n_mels, self.mel_nframes), melsp.min())
        image_tens[0, :, :has_mel_nframes] = melsp[:, :, :has_mel_nframes]

        return image_tens

    def load_item(self, df, idx, w_type):

        sample_wav_file = df['filename'][idx]
        wave, _ = torchaudio.load(sample_wav_file)
        wave = wave.float()

        if w_type==1:
            wave_noise = wave.numpy()
            wave_noise_no_silence = [float(el) for el in wave_noise[0] if abs(el) >= 0.001]
            wave = torch.tensor([wave_noise_no_silence])

        wave_melsp = self.get_same_shape(wave)

        return wave_melsp

    def __getitem__(self, idx):

        wave_speech = self.load_item(self.speech, idx, w_type=0)

        luck = random.random()
        if luck > 0.5:
            wave_noise = self.load_item(self.noise, idx, w_type=1)

            wave_out = wave_speech + wave_noise
            label = 1

        else:
            wave_out = wave_speech
            label = 0

        return wave_out, torch.tensor(label)

In [16]:
test_dataset = TestDataset('./CleanSpeechCustomTest.csv',
                            './NoiseTest.csv')

In [17]:
test_loader = DataLoader(dataset=test_dataset, batch_size=4, 
                         drop_last=True)

In [53]:
vgg_loaded_test_accuracy, vgg_loaded_test_loss = compute_acc_loss(vgg_model, test_loader, loss)
print(f'VGG model test accuracy: {vgg_loaded_test_accuracy:.2f}')
print(f'VGG model test loss: {vgg_loaded_test_loss:.3f}')

VGG model test accuracy: 0.82
VGG model test loss: 0.359


### Custom Test (in Russian) vor VGG Model

In [19]:
class RTestDataset(Dataset):

    def __init__(self, path_speech, path_noise, eps_value=1e-6,\
                 mel_nframes=300, sample_rate=16000, n_fft=480,\
                 hop_length=160, n_mels=40):

        self.speech = pd.read_csv(path_speech)
        self.noise = pd.read_csv(path_noise)
        self.eps_value = eps_value
        self.mel_nframes = mel_nframes
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.mel_spectrogram = T.MelSpectrogram(sample_rate=self.sample_rate,
                                                n_fft=self.n_fft,
                                                hop_length=self.hop_length,
                                                center=True,
                                                pad_mode="reflect",
                                                power=2.0,
                                                norm='slaney',
                                                n_mels=self.n_mels)

    def __len__(self):

        return len(self.speech)

    def get_same_shape(self, waveform):

        melsp = torch.log(self.mel_spectrogram(waveform) + self.eps_value)
        has_mel_nframes = melsp.size(2) if melsp.size(2) < self.mel_nframes\
         else self.mel_nframes

        image_tens = torch.full((1, self.n_mels, self.mel_nframes), melsp.min())
        image_tens[0, :, :has_mel_nframes] = melsp[:, :, :has_mel_nframes]

        return image_tens

    def load_item(self, df, idx, w_type):

        sample_wav_file = df['filename'][idx]
        wave, _ = torchaudio.load(sample_wav_file)
        wave = wave.float()

        if w_type==1:
            wave_noise = wave.numpy()
            wave_noise_no_silence = [float(el) for el in wave_noise[0] if abs(el) >= 0.001]
            wave = torch.tensor([wave_noise_no_silence])

        wave_melsp = self.get_same_shape(wave)

        return wave_melsp

    def __getitem__(self, idx):

        wave_speech = self.load_item(self.speech, idx, w_type=0)

        luck = random.random()
        if luck > 0.5:
            wave_noise = self.load_item(self.noise, idx, w_type=1)

            wave_out = wave_speech + wave_noise
            label = 1

        else:
            wave_out = wave_speech
            label = 0

        return wave_out, torch.tensor(label)

In [20]:
rtest_dataset = RTestDataset('./CleanSpeechCustomTestRus.csv',
                             './NoiseTest.csv')

In [21]:
rtest_loader = DataLoader(dataset=rtest_dataset, batch_size=2, 
                          drop_last=True)

In [58]:
vgg_loaded_test_accuracy, vgg_loaded_test_loss = compute_acc_loss(vgg_model, rtest_loader, loss)
print(f'VGG model test accuracy: {vgg_loaded_test_accuracy:.2f}')
print(f'VGG model test loss: {vgg_loaded_test_loss:.3f}')

VGG model test accuracy: 0.90
VGG model test loss: 0.443


### Testing VGG model on phone recordings (in Russian)

In [15]:
phone_dataset = PhoneDataset('./CleanSpeechCustomTest.csv',
                            './NoiseTest.csv')

In [16]:
phone_loader = DataLoader(dataset=phone_dataset, batch_size=4, 
                          drop_last=True)

In [17]:
from vggNet import VGGModel

vgg_model = VGGModel(1, 'vgg11')
vgg_model.load_state_dict(torch.load('./weights_vgg1.pth'))
vgg_model.type(torch.FloatTensor)

loss = nn.BCELoss().type(torch.FloatTensor)

optimizer = optim.SGD(vgg_model.parameters(), lr=1e-3)
optimizer.load_state_dict(torch.load('./optimizer_vgg1.pth'))

In [18]:
vgg_phone_test_accuracy, vgg_phone_test_loss = compute_acc_loss(vgg_model, phone_loader, loss)
print(f'VGG model test accuracy: {vgg_phone_test_accuracy:.2f}')
print(f'VGG model test loss: {vgg_phone_test_loss:.3f}')

VGG model test accuracy: 0.89
VGG model test loss: 0.150


### ResNet50

In [10]:
from ResNet import ResNet50

model_rn = ResNet50(num_classes=1, device=device)
model_rn.type(torch.cuda.FloatTensor)
model_rn.to(device)

loss = nn.BCELoss().type(torch.cuda.FloatTensor)
optimizer = optim.SGD(model_rn.parameters(), lr=1e-3)

cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, verbose=True)
warmup_scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=20,
                                          after_scheduler=cosine_scheduler)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Adjusting learning rate of group 0 to 1.0000e-03.


In [11]:
loss_history, train_history, val_history = train(model_rn, train_loader,
                                                 val_loader, loss,
                                                 optimizer, warmup_scheduler,
                                                 num_epochs=50)

Epoch 1:   0%|          | 0/48 [00:08<?, ?it/s]


AttributeError: 'ResNet50' object has no attribute 'num_labels'